In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

!rm -rf /kaggle/working/RecSysChallenge2024 && git clone https://{token}@github.com/FrancescoZanella/RecSysChallenge2024.git
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi.git

Cloning into 'RecSysChallenge2024'...
remote: Enumerating objects: 1426, done.
remote: Counting objects: 100% (402/402), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 1426 (delta 206), reused 257 (delta 122), pack-reused 1024
Receiving objects: 100% (1426/1426), 32.39 MiB | 33.13 MiB/s, done.
Resolving deltas: 100% (702/702), done.
Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1519, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1519 (delta 223), reused 220 (delta 212), pack-reused 1264
Receiving objects: 100% (1519/1519), 50.33 MiB | 28.94 MiB/s, done.
Resolving deltas: 100% (909/909), done.


In [2]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 65.3 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [3]:
import sklearn as sk
sk.__version__

'1.4.2'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import gc
import tqdm
from catboost import CatBoostClassifier
from colorama import Fore

In [5]:
%cd /kaggle/working/RecSys_Course_AT_PoliMi
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Evaluation.Evaluator import EvaluatorHoldout

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

/kaggle/working/RecSys_Course_AT_PoliMi


In [6]:
%cd /kaggle/working/RecSysChallenge2024/src
from ebrec.utils._descriptive_analysis import (
    min_max_impression_time_behaviors, 
    min_max_impression_time_history
)
from ebrec.utils._polars import slice_join_dataframes
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    truncate_history,
)
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_USER_COL
)
from ebrec.evaluation.metrics_protocols import *

from polimi.utils._custom import GetMemUsage, PrintColor

/kaggle/working/RecSysChallenge2024/src


In [7]:
articles = pl.read_parquet('/kaggle/input/recommender-systems-challenge-2024/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/kaggle/input/recommender-systems-challenge-2024/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/kaggle/input/recommender-systems-challenge-2024/ebnerd_small/train/history.parquet')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 0.5974


In [10]:
from sklearn.preprocessing import TargetEncoder

def build_target_encoder(X:np.ndarray, y:np.ndarray, smooth='auto', target_type='binary', cv=5):
    encoder = TargetEncoder(smooth=smooth, target_type=target_type, cv=cv).fit(X, y)
    return encoder
    

In [11]:
def build_target_encoder_features(df_features: pl.DataFrame, behaviors: pl.DataFrame, cols=['user_id'], target='labels') -> pl.DataFrame:
    df = behaviors.pipe(create_binary_labels_column, shuffle=True, seed=123)\
        .select(cols + [target])\
        .explode(target)\
        .sort(cols)

    encoder = build_target_encoder(df.select(cols).to_numpy(), df['labels'].to_numpy())
    X_enc = encoder.transform(df_features.select(cols).to_numpy())
    df_features = df_features.with_columns(pl.Series(X_enc[:, i]).alias(f'{col}_target_encoded') for i, col in enumerate(cols))
    
    return df_features, encoder

In [15]:
history_n, encoder = build_target_encoder_features(history_train, behaviors_train, cols=['user_id'])

In [16]:
history_n.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,user_id_target_encoded
u32,list[datetime[μs]],list[f32],list[i32],list[f32],f64
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]",0.137154
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]",0.103281
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]",0.041863
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]",0.123699
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]",0.124067
